In [1]:
import pandas as pd
import numpy as np
import statistics

## Loading metrics

In [2]:
samples = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")
samples["full_name"] = samples["framework"] + "/" +samples["path"]
samples = samples["full_name"].unique()

In [3]:
def loadMetrics(listSamples):
    metrics = dict()
    for sample in listSamples:
        metrics[sample] = pd.read_csv(f"../3-DataMerge/{sample}.csv", index_col=[0,1], na_filter=False)
    return metrics

In [4]:
normalized_metrics_questions = loadMetrics(samples)

## Mann Kendall Test

Null Hypothesis: There is no monotonic trend.

Alternative Hypothesis: There is a trend.

If p-value < 0.05 then null Hypothesis is rejected.

In [5]:
pip install pymannkendall

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pymannkendall as mk

In [7]:
def extractMannKendallTest(metrics, listSamples):
    result = pd.DataFrame(index=listSamples)
    result.index.rename("full_name", inplace=True)
    for sample in listSamples:
        sampleMetric = metrics[sample]
        for metric in sampleMetric.columns.values:
            mkResult = mk.original_test(sampleMetric[metric])
            result.loc[sample, "framework"] = sample.split("/")[0]
            result.loc[sample, metric+" trend"] = mkResult.trend
            result.loc[sample, metric+" pvalue"] = mkResult.p
    return result

In [8]:
mannKendallResult = extractMannKendallTest(normalized_metrics_questions, normalized_metrics_questions.keys())

In [9]:
mannKendallResult

,framework,JAVA trend,JAVA pvalue,CLASSES trend,CLASSES pvalue,PKG trend,PKG pvalue,BWR trend,BWR pvalue,SAR trend,...,LCOM trend,LCOM pvalue,CBO trend,CBO pvalue,RFC trend,RFC pvalue,NIV trend,NIV pvalue,questions trend,questions pvalue
full_name,,,,,,,,,,,,,,,,,,,,,
googlesamples/io2014-codelabs,googlesamples,no trend,4.701008e-01,no trend,4.701008e-01,no trend,1.000000,no trend,0.470101,no trend,...,no trend,0.470101,no trend,1.000000e+00,no trend,0.470101,no trend,4.701008e-01,no trend,3.710934e-01
googlesamples/android-play-publisher-api,googlesamples,increasing,1.234535e-07,increasing,1.234535e-07,no trend,1.000000,no trend,1.000000,increasing,...,no trend,1.000000,increasing,1.234535e-07,no trend,1.000000,no trend,1.000000e+00,increasing,0.000000e+00
googlesamples/androidtv-sample-inputs,googlesamples,increasing,0.000000e+00,increasing,0.000000e+00,increasing,0.000000,decreasing,0.003678,increasing,...,increasing,0.000909,increasing,1.817646e-04,increasing,0.000000,no trend,9.090662e-02,increasing,0.000000e+00
googlesamples/android-AppUsageStatistics,googlesamples,no trend,1.000000e+00,no trend,1.000000e+00,no trend,1.000000,no trend,1.000000,decreasing,...,no trend,1.000000,increasing,1.398555e-03,no trend,1.000000,no trend,1.000000e+00,increasing,2.220446e-16
googlesamples/android-testing-templates,googlesamples,no trend,1.000000e+00,no trend,1.000000e+00,no trend,1.000000,no trend,1.000000,no trend,...,no trend,1.000000,no trend,1.042497e-01,no trend,1.000000,no trend,1.000000e+00,increasing,2.723113e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spring-cloud-samples/spring-cloud-contract-samples,spring-cloud-samples,increasing,0.000000e+00,increasing,0.000000e+00,increasing,0.000000,increasing,0.000002,decreasing,...,decreasing,0.000000,increasing,7.993606e-15,increasing,0.000000,decreasing,7.001733e-12,increasing,0.000000e+00
spring-cloud-samples/stub-runner-boot,spring-cloud-samples,no trend,1.000000e+00,no trend,1.000000e+00,no trend,1.000000,no trend,1.000000,no trend,...,no trend,1.000000,no trend,1.000000e+00,no trend,1.000000,no trend,1.000000e+00,increasing,1.766470e-06
spring-cloud-samples/sample-zuul-filters,spring-cloud-samples,no trend,1.366410e-01,no trend,1.366410e-01,no trend,1.000000,no trend,0.136641,no trend,...,no trend,1.000000,no trend,1.366410e-01,no trend,0.136641,no trend,1.000000e+00,increasing,3.353436e-03


In [10]:
mannKendallResult.to_csv("mann_kendall_results.csv")

## Normality Test (Shapiro-Wilk)

Null Hyphotesis: The population has normal distribution

Alternative Hyphotesis: The population has not normal distribution

If p-value < 0.05 then null hyphotesis is rejected

## Correlation Test

Null Hypothesis: there is no correlation between data

Alternative Hypothesis: there is a correlation

If p-value < 0.05 then null Hypothesis is rejected.

If the metric is normalized we will apply the Pearson Test, otherwise we will apply Spearman Test

In [11]:
from scipy import stats

In [12]:
def extractCorrelationTest(metrics, listSamples):
    result = pd.DataFrame(index=listSamples)
    result.index.rename("full_name", inplace=True)
    for sample in listSamples:
        sampleMetric = metrics[sample]
        for metric in sampleMetric.columns.values:
            print(f"{sample} -> {metric}")
            #test if the set has normal distribution
            shapiroPvalue = stats.shapiro(sampleMetric[metric]).pvalue
            if(shapiroPvalue < 0.05):
                correlationResult = stats.spearmanr(sampleMetric["questions"], sampleMetric[metric], nan_policy="omit")    
            else:
                correlationResult = stats.pearsonr(sampleMetric["questions"], sampleMetric[metric])
            result.loc[sample, metric+" correlation"] = correlationResult[0]
            result.loc[sample, metric+" pvalue"] = correlationResult[1]
            result.loc[sample, "framework"] = sample.split("/")[0]

    return result

In [13]:
correlationResult = extractCorrelationTest(normalized_metrics_questions, normalized_metrics_questions.keys())

googlesamples/io2014-codelabs -> JAVA
googlesamples/io2014-codelabs -> CLASSES
googlesamples/io2014-codelabs -> PKG
googlesamples/io2014-codelabs -> BWR
googlesamples/io2014-codelabs -> SAR
googlesamples/io2014-codelabs -> FANIN
googlesamples/io2014-codelabs -> FANOUT
googlesamples/io2014-codelabs -> CC
googlesamples/io2014-codelabs -> CCM
googlesamples/io2014-codelabs -> CCS
googlesamples/io2014-codelabs -> EC
googlesamples/io2014-codelabs -> KNTS
googlesamples/io2014-codelabs -> NPATH
googlesamples/io2014-codelabs -> NPLOG
googlesamples/io2014-codelabs -> PL
googlesamples/io2014-codelabs -> LOC
googlesamples/io2014-codelabs -> DLOC
googlesamples/io2014-codelabs -> ELOC
googlesamples/io2014-codelabs -> CLOC
googlesamples/io2014-codelabs -> BL
googlesamples/io2014-codelabs -> NV
googlesamples/io2014-codelabs -> AM
googlesamples/io2014-codelabs -> NPRM
googlesamples/io2014-codelabs -> NPM
googlesamples/io2014-codelabs -> PM
googlesamples/io2014-codelabs -> LM
googlesamples/io2014-codela

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


googlesamples/android-play-publisher-api -> BL
googlesamples/android-play-publisher-api -> NV
googlesamples/android-play-publisher-api -> AM
googlesamples/android-play-publisher-api -> NPRM
googlesamples/android-play-publisher-api -> NPM
googlesamples/android-play-publisher-api -> PM
googlesamples/android-play-publisher-api -> LM
googlesamples/android-play-publisher-api -> STMT
googlesamples/android-play-publisher-api -> DSTMT
googlesamples/android-play-publisher-api -> ESTMT
googlesamples/android-play-publisher-api -> CCR
googlesamples/android-play-publisher-api -> WMC
googlesamples/android-play-publisher-api -> NOC
googlesamples/android-play-publisher-api -> DIT
googlesamples/android-play-publisher-api -> LCOM
googlesamples/android-play-publisher-api -> CBO
googlesamples/android-play-publisher-api -> RFC
googlesamples/android-play-publisher-api -> NIV
googlesamples/android-play-publisher-api -> questions
googlesamples/androidtv-sample-inputs -> JAVA
googlesamples/androidtv-sample-inp

googlesamples/android-vision -> CC
googlesamples/android-vision -> CCM
googlesamples/android-vision -> CCS
googlesamples/android-vision -> EC
googlesamples/android-vision -> KNTS
googlesamples/android-vision -> NPATH
googlesamples/android-vision -> NPLOG
googlesamples/android-vision -> PL
googlesamples/android-vision -> LOC
googlesamples/android-vision -> DLOC
googlesamples/android-vision -> ELOC
googlesamples/android-vision -> CLOC
googlesamples/android-vision -> BL
googlesamples/android-vision -> NV
googlesamples/android-vision -> AM
googlesamples/android-vision -> NPRM
googlesamples/android-vision -> NPM
googlesamples/android-vision -> PM
googlesamples/android-vision -> LM
googlesamples/android-vision -> STMT
googlesamples/android-vision -> DSTMT
googlesamples/android-vision -> ESTMT
googlesamples/android-vision -> CCR
googlesamples/android-vision -> WMC
googlesamples/android-vision -> NOC
googlesamples/android-vision -> DIT
googlesamples/android-vision -> LCOM
googlesamples/android

aws-samples/eb-java-scorekeep -> PM
aws-samples/eb-java-scorekeep -> LM
aws-samples/eb-java-scorekeep -> STMT
aws-samples/eb-java-scorekeep -> DSTMT
aws-samples/eb-java-scorekeep -> ESTMT
aws-samples/eb-java-scorekeep -> CCR
aws-samples/eb-java-scorekeep -> WMC
aws-samples/eb-java-scorekeep -> NOC
aws-samples/eb-java-scorekeep -> DIT
aws-samples/eb-java-scorekeep -> LCOM
aws-samples/eb-java-scorekeep -> CBO
aws-samples/eb-java-scorekeep -> RFC
aws-samples/eb-java-scorekeep -> NIV
aws-samples/eb-java-scorekeep -> questions
aws-samples/amazon-ecs-java-microservices -> JAVA
aws-samples/amazon-ecs-java-microservices -> CLASSES
aws-samples/amazon-ecs-java-microservices -> PKG
aws-samples/amazon-ecs-java-microservices -> BWR
aws-samples/amazon-ecs-java-microservices -> SAR
aws-samples/amazon-ecs-java-microservices -> FANIN
aws-samples/amazon-ecs-java-microservices -> FANOUT
aws-samples/amazon-ecs-java-microservices -> CC
aws-samples/amazon-ecs-java-microservices -> CCM
aws-samples/amazon-ecs

...

googlearchive/android-ScopedDirectoryAccess -> STMT
googlearchive/android-ScopedDirectoryAccess -> DSTMT
googlearchive/android-ScopedDirectoryAccess -> ESTMT
googlearchive/android-ScopedDirectoryAccess -> CCR
googlearchive/android-ScopedDirectoryAccess -> WMC
googlearchive/android-ScopedDirectoryAccess -> NOC
googlearchive/android-ScopedDirectoryAccess -> DIT
googlearchive/android-ScopedDirectoryAccess -> LCOM
googlearchive/android-ScopedDirectoryAccess -> CBO
googlearchive/android-ScopedDirectoryAccess -> RFC
googlearchive/android-ScopedDirectoryAccess -> NIV
googlearchive/android-ScopedDirectoryAccess -> questions
googlearchive/leanback-showcase -> JAVA
googlearchive/leanback-showcase -> CLASSES
googlearchive/leanback-showcase -> PKG
googlearchive/leanback-showcase -> BWR
googlearchive/leanback-showcase -> SAR
googlearchive/leanback-showcase -> FANIN
googlearchive/leanback-showcase -> FANOUT
googlearchive/leanback-showcase -> CC
googlearchive/leanback-showcase -> CCM
googlearchive/le

googlearchive/android-NotificationChannels -> NPLOG
googlearchive/android-NotificationChannels -> PL
googlearchive/android-NotificationChannels -> LOC
googlearchive/android-NotificationChannels -> DLOC
googlearchive/android-NotificationChannels -> ELOC
googlearchive/android-NotificationChannels -> CLOC
googlearchive/android-NotificationChannels -> BL
googlearchive/android-NotificationChannels -> NV
googlearchive/android-NotificationChannels -> AM
googlearchive/android-NotificationChannels -> NPRM
googlearchive/android-NotificationChannels -> NPM
googlearchive/android-NotificationChannels -> PM
googlearchive/android-NotificationChannels -> LM
googlearchive/android-NotificationChannels -> STMT
googlearchive/android-NotificationChannels -> DSTMT
googlearchive/android-NotificationChannels -> ESTMT
googlearchive/android-NotificationChannels -> CCR
googlearchive/android-NotificationChannels -> WMC
googlearchive/android-NotificationChannels -> NOC
googlearchive/android-NotificationChannels ->

googlearchive/android-DarkTheme -> NPATH
googlearchive/android-DarkTheme -> NPLOG
googlearchive/android-DarkTheme -> PL
googlearchive/android-DarkTheme -> LOC
googlearchive/android-DarkTheme -> DLOC
googlearchive/android-DarkTheme -> ELOC
googlearchive/android-DarkTheme -> CLOC
googlearchive/android-DarkTheme -> BL
googlearchive/android-DarkTheme -> NV
googlearchive/android-DarkTheme -> AM
googlearchive/android-DarkTheme -> NPRM
googlearchive/android-DarkTheme -> NPM
googlearchive/android-DarkTheme -> PM
googlearchive/android-DarkTheme -> LM
googlearchive/android-DarkTheme -> STMT
googlearchive/android-DarkTheme -> DSTMT
googlearchive/android-DarkTheme -> ESTMT
googlearchive/android-DarkTheme -> CCR
googlearchive/android-DarkTheme -> WMC
googlearchive/android-DarkTheme -> NOC
googlearchive/android-DarkTheme -> DIT
googlearchive/android-DarkTheme -> LCOM
googlearchive/android-DarkTheme -> CBO
googlearchive/android-DarkTheme -> RFC
googlearchive/android-DarkTheme -> NIV
googlearchive/and

spring-cloud-samples/processor -> FANOUT
spring-cloud-samples/processor -> CC
spring-cloud-samples/processor -> CCM
spring-cloud-samples/processor -> CCS
spring-cloud-samples/processor -> EC
spring-cloud-samples/processor -> KNTS
spring-cloud-samples/processor -> NPATH
spring-cloud-samples/processor -> NPLOG
spring-cloud-samples/processor -> PL
spring-cloud-samples/processor -> LOC
spring-cloud-samples/processor -> DLOC
spring-cloud-samples/processor -> ELOC
spring-cloud-samples/processor -> CLOC
spring-cloud-samples/processor -> BL
spring-cloud-samples/processor -> NV
spring-cloud-samples/processor -> AM
spring-cloud-samples/processor -> NPRM
spring-cloud-samples/processor -> NPM
spring-cloud-samples/processor -> PM
spring-cloud-samples/processor -> LM
spring-cloud-samples/processor -> STMT
spring-cloud-samples/processor -> DSTMT
spring-cloud-samples/processor -> ESTMT
spring-cloud-samples/processor -> CCR
spring-cloud-samples/processor -> WMC
spring-cloud-samples/processor -> NOC
spr

spring-cloud-samples/brewery -> BL
spring-cloud-samples/brewery -> NV
spring-cloud-samples/brewery -> AM
spring-cloud-samples/brewery -> NPRM
spring-cloud-samples/brewery -> NPM
spring-cloud-samples/brewery -> PM
spring-cloud-samples/brewery -> LM
spring-cloud-samples/brewery -> STMT
spring-cloud-samples/brewery -> DSTMT
spring-cloud-samples/brewery -> ESTMT
spring-cloud-samples/brewery -> CCR
spring-cloud-samples/brewery -> WMC
spring-cloud-samples/brewery -> NOC
spring-cloud-samples/brewery -> DIT
spring-cloud-samples/brewery -> LCOM
spring-cloud-samples/brewery -> CBO
spring-cloud-samples/brewery -> RFC
spring-cloud-samples/brewery -> NIV
spring-cloud-samples/brewery -> questions
spring-cloud-samples/github-eureka -> JAVA
spring-cloud-samples/github-eureka -> CLASSES
spring-cloud-samples/github-eureka -> PKG
spring-cloud-samples/github-eureka -> BWR
spring-cloud-samples/github-eureka -> SAR
spring-cloud-samples/github-eureka -> FANIN
spring-cloud-samples/github-eureka -> FANOUT
spri

In [14]:
correlationResult.to_csv("correlation_results.csv")